In [1]:
#python librairies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from IPython.core.display import display, HTML


C:\Users\Mohaw\AppData\Local\Temp\ipykernel_11956\3127415297.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#Import the cleaned dataset

#Importation data
data = pd.read_csv("StackOverflow_cleaned.csv", sep=";",
                   index_col=0,
                   converters={"Title": literal_eval,
                               "Body": literal_eval,
                               "Tags": literal_eval})
data.head(5)


,Title,Body,Score,Tags
Id,,,,
4795928,"[check, element, exists]","[possible, duplicate, exists, function, jquery...",31,"[javascript, jquery]"
4796305,"[internet, explorer, send, http, post, body, a...","[able, reliably, recreate, following, scenario...",115,[javascript]
4797373,"[eval, point]","[official, documentation, regarding, function,...",6,[php]
4797418,"[tracking, object, allocation, python]","[method, override, allow, use, print, statemen...",10,[python]
4797444,"[weird, error, installing, android, app]","[intellij, idea, exported, signed, application...",7,[android]


In [3]:
data['text'] = data['Title'] + data['Body']

In [4]:
# transform target tags
mlb = MultiLabelBinarizer()
mlb.fit(data.Tags)
y_mlb = mlb.transform(data.Tags)
print("Number of Tags:", len(mlb.classes_))
#
# data split
X_train, X_test, y_train, y_test = train_test_split(data.text,\
                    y_mlb, test_size=0.25, random_state=42)


Number of Tags: 50


In [8]:

#Piple of the final modèle
# define model
# tfidf features
final_model = Pipeline([('tfidf', TfidfVectorizer(analyzer="word",
                             max_df=.98,
                             min_df= 2,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)),
                        ('lr', OneVsRestClassifier(LogisticRegression()))])

#fit model
final_model.fit(X_train,y_train)

#prediction and score
y_score = final_model.predict_proba(X_test)
print('Score ROC_AUC:', metrics.roc_auc_score(y_test, y_score,multi_class="ovr"))

KeyboardInterrupt: 

In [ ]:
from pickle import dump

#Modèle serialisation
dump(final_model, open("final_model.pkl","wb"))
dump(mlb, open("mlb.pkl","wb"))

In [ ]:
from pickle import load
final_model=load(open("final_model.pkl","rb"))
mlb=load(open("mlb.pkl","rb"))

In [ ]:
# Example with one text
text = ['python','model','c#','class','C++']
#threshold for labels with scores, fitted from the ROC 
threshold = 0.11 
y_pred = final_model.predict_proba(np.array([text,]))
tag = mlb.inverse_transform((y_pred>threshold)*1)

In [ ]:
print('Tags proposés:', tag)

In [ ]:
model.py — Contient le code permettant au modèle d’apprentissage automatique de prédire les ventes du troisième mois en fonction des ventes des deux premiers mois.
app.py — Il contient les API Flask qui reçoivent les détails des ventes via une interface graphique ou des appels d’API, calculent la valeur prédite en fonction de notre modèle et la renvoient.
request.py — Cette option utilise le module requests pour appeler les API définies dans app.py et affiche la valeur renvoyée.
HTML/CSS — Contient le modèle HTML et le style CSS pour permettre à l’utilisateur d’entrer les détails des ventes et affiche les ventes prévues au cours du troisième mois.

Où est le code?
Sans trop tarder, commençons par le code. Le projet complet sur github peut être trouvé ici.

Commençons par créer le frontal à l’aide de HTML pour que l’utilisateur puisse entrer les valeurs. Il y a trois champs qui doivent être remplis par l’utilisateur: taux d’intérêt, ventes au premier mois et ventes au deuxième mois.

In [ ]:
<!DOCTYPE html>
<html >
<head>
  <meta charset="UTF-8">
  <title>Deployment Tutorial 1</title>
  <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Arimo' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Hind:300' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'>
<link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
  
</head>

<body style="background: #000;">
 <div class="login">
<h1>Sales Forecasting</h1>

     <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
    <input type="text" name="rate" placeholder="rate" required="required" />
        <input type="text" name="sales in first month" placeholder="sales in first month" required="required" />
<input type="text" name="sales in second month" placeholder="sales in second month" required="required" />
        <button type="submit" class="btn btn-primary btn-block btn-large">Predict sales in third month</button>
    </form>

   <br>
   <br>
   {{ prediction_text }}

 </div>
</body>
</html>

In [ ]:
@import url(https://fonts.googleapis.com/css?family=Open+Sans);

html { width: 100%; height:100%; overflow:hidden; }

body { 
   width: 100%;
   height:100%;
   font-family: 'Helvetica';
   background: #000;
   color: #fff; 
     font-size: 24px;
    text-align:center;
   letter-spacing:1.4px;

}
.login { 
    position: absolute;
    top: 40%;
   left: 50%;
     margin: -150px 0 0 -150px;
   width:400px;
   height:400px;
}

.login h1 { color: #fff; text-shadow: 0 0 10px rgba(0,0,0,0.3); letter-spacing:1px; text-align:center; }

input { 
   width: 100%; 
   margin-bottom: 10px; 
   background: rgba(0,0,0,0.3);
   border: none;
   outline: none;
   padding: 10px;
   font-size: 13px;
   color: #fff;
   text-shadow: 1px 1px 1px rgba(0,0,0,0.3);
   border: 1px solid rgba(0,0,0,0.3);
   border-radius: 4px;
   box-shadow: inset 0 -5px 45px rgba(100,100,100,0.2), 0 1px 1px rgba(255,255,255,0.2);
   -webkit-transition: box-shadow .5s ease;
  -moz-transition: box-shadow .5s ease;
   -o-transition: box-shadow .5s ease;
    -ms-transition: box-shadow .5s ease;
    transition: box-shadow .5s ease;
}